In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [ ]:
graph_def = tf.GraphDef()
model = "yolov3-tiny.pb"
with tf.gfile.FastGFile(model, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

In [ ]:
sess = tf.Session()

In [ ]:
input_ = sess.graph.get_tensor_by_name('tinyyolov3/net1:0')

In [ ]:
output_1 = sess.graph.get_tensor_by_name('tinyyolov3/convolutional10/BiasAdd:0')
output_2 = sess.graph.get_tensor_by_name('tinyyolov3/convolutional13/BiasAdd:0')

In [ ]:
img = cv2.imread("New44.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
img = cv2.resize(img, (416,416))

In [ ]:
img = img / 255.0

In [ ]:
img = img.reshape(1, 416, 416,3)

In [ ]:
ans_1, ans_2 = sess.run([output_1,output_2], feed_dict={input_ : img})

In [ ]:
anchors = [[(10,14),  (23,27),  (37,58)], [(81,82),  (135,169),  (344,319)]]

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
ans = ans_1.reshape(1,13,13,3,-1)

In [ ]:
def class_probabilities(output_layer, objectness_threshold):
    output_layer[..., 0:2] = sigmoid(output_layer[..., 0:2])
    output_layer[..., 4:] = sigmoid(output_layer[..., 4:])
    output_layer[..., 5:]  = output_layer[..., 4][..., np.newaxis] * output_layer[..., 5:]
    output_layer[..., 5:] *= output_layer[..., 5:] > objectness_threshold
    return output_layer

In [ ]:
output_layer_1 = class_probabilities(output_layer = ans_1 , objectness_threshold = 0.1 )
output_layer_1 = output_layer_1.reshape(1,13,13,3,-1)

In [ ]:
output_layer_2 = class_probabilities(output_layer = ans_2, objectness_threshold = 0.1 )
output_layer_2 = output_layer_2.reshape(1,26,26,3,-1)

In [ ]:
output_layer_2.shape

In [ ]:
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        self.objness = objness
        self.classes = classes
        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)

        return self.label

    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]

        return self.score

In [ ]:
def boxes_filter_scale(output_layer, anchor, threshold, image_height, image_width):
    boxes = []
    useful_boxes = 0
    num_row, num_col = output_layer.shape[0:2]
    num_boxes = len(anchor)
    for row in range(num_row):
        for col in range(num_col):
            for box in range(num_boxes):
                if output_layer[row][col][box][4] >= threshold:
                    useful_boxes = useful_boxes+1
                    x,y,h,w = output_layer[row][col][box][:4]
                    x = col + x 
                    x = x / num_col
                    y = row + y
                    y = y / num_row
                    w = anchor[box][0] * np.exp(w) / image_width 
                    h = anchor[box][1] * np.exp(h) / image_height
                    x1, y1 = x - w/2, y - h/2
                    x2, y2 = x + w/2, y + h/2
                    
                    if x1 < 0 :
                        x1 = 0
                    if y1 < 0 :
                        y1 = 0
                    if x2 > 1 :
                        x2 = 1
                    if y2 > 1 :
                        y2 = 1
                        
                    classes = output_layer[row][col][box][5:]
                    objectness = output_layer[row][col][box][4]
                    box = BoundBox(x1, y1, x2, y2, objectness, classes)
                    boxes.append(box)
    total_boxes = num_col * num_row * num_boxes
    print("Out of {} , Num Boxes Filtered : {} ".format(total_boxes, useful_boxes))
    return boxes

In [ ]:
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
    new_w, new_h = net_w, net_h
    for i in range(len(boxes)):
        x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)

In [ ]:
scaled_boxes = []
for batch in range(output_layer_1.shape[0]):
    print("Batch : ", str(batch+1))
    output_layer_1_boxes = boxes_filter_scale(output_layer_1[batch], anchors[0], 0.1, 416, 416)
    output_layer_2_boxes = boxes_filter_scale(output_layer_2[batch], anchors[1], 0.1, 416, 416)
    scaled_boxes.append([*output_layer_1_boxes, *output_layer_2_boxes])
    print("**************************************")

In [ ]:
output_layer_1[0].shape

In [ ]:
correct_yolo_boxes(scaled_boxes[0], 416, 416, 416, 416)

In [ ]:
scaled_boxes

In [ ]:
len(scaled_boxes[0])

In [ ]:
def py_nms(boxes, scores, max_boxes=50, iou_thresh=0.4):

    assert boxes.shape[1] == 4 and len(scores.shape) == 1

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= iou_thresh)[0]
        order = order[inds + 1]

    return keep

In [ ]:
def cpu_nms(boxes, scores, num_classes, max_boxes=50, score_thresh=0.5, iou_thresh=0.5):

    boxes = boxes.reshape(-1, 4)
    scores = scores.reshape(-1, num_classes)
    picked_boxes, picked_score, picked_label = [], [], []

    for i in range(num_classes):
        indices = np.where(scores[:,i] >= score_thresh)
        filter_boxes = boxes[indices]
        filter_scores = scores[:,i][indices]
        if len(filter_boxes) == 0: 
            continue
        indices = py_nms(filter_boxes, filter_scores, max_boxes, iou_thresh)
        picked_boxes.append(filter_boxes[indices])
        picked_score.append(filter_scores[indices])
        picked_label.append(np.ones(len(indices), dtype='int32')*i)
    if len(picked_boxes) == 0: 
        return None, None, None

    boxes = np.concatenate(picked_boxes, axis=0)
    score = np.concatenate(picked_score, axis=0)
    label = np.concatenate(picked_label, axis=0)

    return boxes, score, label

In [ ]:
dets = []
for box in scaled_boxes[0]:
    dets.append([box.xmin, box.ymin, box.xmax, box.ymax, box.objness, *box.classes])
dets = np.array(dets)

In [ ]:
dets

In [ ]:
boxes, score, label = cpu_nms(dets[:, 0:4], dets[:,5:], 3, 50, 0.2, 0.4)

In [ ]:
imgs = img[0]
for box,l in zip(boxes, label):
    
    assert box[0] < box[2]
    assert box[1] < box[3]
    
    if l ==  0:
        r,g,b = 255,255,255
        print(box, l)
    elif l == 1:
        r,g,b = 0,255,0
        print(box, l)
    elif l == 2:
        r,g,b = 0,0,255
        print(box, l)
    else :
        r,g,b = 255,255,0
    cv2.rectangle(imgs,(int(box[0]), int(box[1])),(int(box[2]), int(box[3])),(r,g,b),2)

plt.figure(figsize=(16,8))
imgplot = plt.imshow(imgs)
plt.show()
print(len(boxes))